In [1]:
import gymnasium as gym
import highway_env

%load_ext tensorboard
import sys
import gymnasium as gym
import highway_env
from stable_baselines3 import DQN

from rl_agents.agents.common.factory import agent_factory

# Visualisation
import sys
from tqdm.notebook import trange
sys.path.insert(0, '/home/zach/HighwayEnv/scripts')
from utils import record_videos, show_videos

2024-01-23 18:42:30.917067: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 18:42:30.943220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 18:42:30.943244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 18:42:30.943922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 18:42:30.948966: I tensorflow/core/platform/cpu_feature_guar

In [2]:
%tensorboard --logdir "llm_dqn"

In [8]:
model = DQN('MlpPolicy', 'llm-v0',
                policy_kwargs=dict(net_arch=[256, 256]),
                learning_rate=5e-4,
                buffer_size=15000,
                learning_starts=200,
                batch_size=128,
                gamma=0.8,
                train_freq=1,
                gradient_steps=1,
                target_update_interval=50,
                exploration_fraction=0.7,
                verbose=1,
                tensorboard_log='llm_dqn/')
model.learn(int(1e5))


Using cuda device
Creating environment from the given name 'llm-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to llm_dqn/DQN_6


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.5      |
|    ep_rew_mean      | 1.98     |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1        |
|    time_elapsed     | 10       |
|    total_timesteps  | 14       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.88     |
|    ep_rew_mean      | 1.45     |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1        |
|    time_elapsed     | 17       |
|    total_timesteps  | 23       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.67     |
|    ep_rew_mean      | 1.3      |
|    exploration_rate | 1        |
| time/               |          |
|    episodes       

In [4]:
from stable_baselines3.common.callbacks import BaseCallback

class FailureAnalysisCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(FailureAnalysisCallback, self).__init__(verbose)
        self.failures = []

    def _on_step(self) -> bool:
        # Access local variables from the training loop
        info = self.locals['infos']
        done = self.locals['dones']
        
        # Check if the episode ended in a failure
        if done and 'crashed' in info:
            obs = self.locals['obs']
            action = self.locals['actions']
            reward = self.locals['rewards']
            next_obs = self.locals['new_obs']
            self.failures.append((obs, action, reward, next_obs, info))
        
        return True

    def _on_training_end(self) -> None:
        # Output the collected failure information at the end of training
        with open('failures.txt', 'w') as file:
            for failure in self.failures:
                file.write(f"{failure}\n")


In [5]:
callback = FailureAnalysisCallback()
model.learn(int(2e4), callback=callback)


Logging to llm_dqn/DQN_4
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.75     |
|    ep_rew_mean      | 1.28     |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1        |
|    time_elapsed     | 8        |
|    total_timesteps  | 11       |
----------------------------------


KeyboardInterrupt: 

In [ ]:
model.save('llm-v0_model')

In [9]:
env = gym.make('llm-v0', render_mode='rgb_array')
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()

/home/zach/.conda/envs/llmav/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/zach/HighwayEnv/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Moviepy - Building video /home/zach/HighwayEnv/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/zach/HighwayEnv/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/zach/HighwayEnv/videos/rl-video-episode-0.mp4


Moviepy - Building video /home/zach/HighwayEnv/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /home/zach/HighwayEnv/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/zach/HighwayEnv/videos/rl-video-episode-1.mp4


Moviepy - Building video /home/zach/HighwayEnv/videos/rl-video-episode-2.mp4.
Moviepy - Writing video /home/zach/HighwayEnv/videos/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/zach/HighwayEnv/videos/rl-video-episode-2.mp4
